In [7]:
import yfinance as yf
import numpy as np
from deap import base, creator, tools, algorithms
import random
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Indian stocks 
indian_stocks = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS',
    'ICICIBANK.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'ITC.NS', 'ASIANPAINT.NS',
    'AXISBANK.NS', 'KOTAKBANK.NS', 'MARUTI.NS', 'LT.NS', 'BAJFINANCE.NS',
    'SUNPHARMA.NS', 'TITAN.NS', 'NESTLEIND.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'
]

# Randomly select 10 stocks
selected_stocks = random.sample(indian_stocks, 10)
print("Selected stocks:", selected_stocks)

# Download historical data (3 years)
end_date = datetime(2025, 5, 22)
start_date = end_date - timedelta(days=3*365)
data = yf.download(selected_stocks, start=start_date, end=end_date)['Close']

# Calculate daily returns and clean outliers
returns = data.pct_change().dropna()
print(f"Returns shape: {returns.shape}")
if returns.empty:
    raise ValueError("Returns DataFrame is empty after dropna. Insufficient data for selected stocks.")

# Ensure returns are numeric
if not returns.select_dtypes(include=[np.number]).columns.equals(returns.columns):
    raise ValueError("Returns DataFrame contains non-numeric columns.")

# Clip outliers column-wise
try:
    returns = returns.apply(lambda x: x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99)))
except Exception as e:
    print(f"Error in clipping returns: {e}")
    raise

# Verify returns after clipping
if returns.isna().any().any():
    print("Warning: NaN values detected after clipping.")
    returns = returns.dropna()

# Convert to numpy array for scenario-based optimization
returns_matrix = returns.values  # T x N matrix (scenarios x assets)
T, N = returns_matrix.shape

print(f"Number of scenarios (T): {T}")
print(f"Number of assets (N): {N}")

mean_returns = returns.mean() * 252  # Annualized mean returns
cov_matrix = returns.cov() * 252    # Annualized covariance matrix

# Print mean returns to check feasibility
print("\nAnnualized Mean Returns:")
for stock, ret in zip(selected_stocks, mean_returns):
    print(f"{stock}: {ret:.4f}")

# Reliability-based portfolio parameters - ADJUSTED FOR FEASIBILITY
d = 0.03                  # Target return (3% annualized) - more realistic
beta_1 = 0.80            # Reliability level (80% probability) - more achievable
alpha = 0.05             # VaR confidence level (5%)
w_max = 0.4              # Maximum weight per asset
w_min = 0.0              # Minimum weight per asset

# Convert daily target return
d_daily = d / 252  # Daily target return

print(f"\nOptimization Parameters:")
print(f"Target Annual Return: {d:.3f}")
print(f"Daily Target Return: {d_daily:.6f}")
print(f"Reliability Level: {beta_1:.3f}")

# CVaR calculation functions
def calculate_cvar(weights, returns_matrix, alpha=0.05):
    """Calculate Conditional Value at Risk (CVaR)"""
    portfolio_returns = np.dot(returns_matrix, weights)
    var_level = np.percentile(portfolio_returns, alpha * 100)
    cvar = np.mean(portfolio_returns[portfolio_returns <= var_level])
    return -cvar  # Return positive value (loss)

def calculate_var(weights, returns_matrix, alpha=0.05):
    """Calculate Value at Risk (VaR)"""
    portfolio_returns = np.dot(returns_matrix, weights)
    var_level = np.percentile(portfolio_returns, alpha * 100)
    return -var_level  # Return positive value (loss)

def calculate_reliability_constraint(weights, returns_matrix, target_return):
    """Calculate probability that portfolio return >= target return"""
    portfolio_returns = np.dot(returns_matrix, weights)
    prob_meet_target = np.mean(portfolio_returns >= target_return)
    return prob_meet_target

def calculate_expected_return(weights, returns_matrix):
    """Calculate expected daily return"""
    portfolio_returns = np.dot(returns_matrix, weights)
    return np.mean(portfolio_returns)

# Enhanced reliability-based objective function
def reliability_objective_fixed(individual):
    """
    Fixed reliability-based objective function with proper constraint handling
    """
    weights = np.array(individual)
    
    # Normalize weights properly
    weights = np.maximum(weights, w_min)
    weights = np.minimum(weights, w_max)
    total_weight = np.sum(weights)
    if total_weight > 0:
        weights = weights / total_weight
    else:
        weights = np.ones(N) / N
    
    # Calculate portfolio metrics
    portfolio_returns = np.dot(returns_matrix, weights)
    expected_daily_return = np.mean(portfolio_returns)
    
    # Primary objective: Maximize expected return (converted to minimization)
    objective = -expected_daily_return * 252  # Negative for minimization
    
    # Calculate constraints
    reliability_prob = calculate_reliability_constraint(weights, returns_matrix, d_daily)
    cvar = calculate_cvar(weights, returns_matrix, alpha)
    
    # Penalty system with different weights
    penalty = 0
    
    # STRONG penalty for reliability constraint violation
    if reliability_prob < beta_1:
        penalty += 10000 * (beta_1 - reliability_prob) ** 2
    
    # Moderate penalty for not meeting minimum return
    if expected_daily_return < d_daily:
        penalty += 1000 * (d_daily - expected_daily_return) ** 2
    
    # Light penalty for weight constraint violations
    weight_sum_error = abs(np.sum(weights) - 1.0)
    if weight_sum_error > 1e-6:
        penalty += 100 * weight_sum_error
    
    weight_bound_violation = (np.sum(np.maximum(0, weights - w_max)) + 
                             np.sum(np.maximum(0, w_min - weights)))
    if weight_bound_violation > 0:
        penalty += 100 * weight_bound_violation
    
    # Add CVaR as secondary objective (scaled down)
    penalty += 0.1 * cvar
    
    total_objective = objective + penalty
    return (total_objective,)

# Multi-objective version for better exploration
def multi_objective_reliability(individual):
    """Multi-objective: minimize CVaR and maximize return with constraints"""
    weights = np.array(individual)
    
    # Normalize weights
    weights = np.maximum(weights, w_min)
    weights = np.minimum(weights, w_max)
    if np.sum(weights) > 0:
        weights = weights / np.sum(weights)
    else:
        weights = np.ones(N) / N
    
    # Calculate metrics
    portfolio_returns = np.dot(returns_matrix, weights)
    expected_return = np.mean(portfolio_returns) * 252
    cvar = calculate_cvar(weights, returns_matrix, alpha)
    reliability_prob = calculate_reliability_constraint(weights, returns_matrix, d_daily)
    
    # Penalties for constraint violations
    penalty = 0
    if reliability_prob < beta_1:
        penalty += 5000 * (beta_1 - reliability_prob)
    if expected_return < d:
        penalty += 1000 * (d - expected_return)
    
    # Multi-objective: minimize CVaR, maximize return
    obj1 = cvar + penalty  # Minimize risk
    obj2 = -expected_return + penalty  # Maximize return (minimize negative return)
    
    return obj1, obj2

# Generate feasible initial weights
def generate_feasible_weights():
    """Generate weights that are more likely to satisfy constraints"""
    # Start with better-performing stocks getting higher weights
    stock_performance = np.array([np.mean(returns_matrix[:, i]) for i in range(N)])
    
    # Create weights biased toward better performers
    if np.max(stock_performance) > np.min(stock_performance):
        normalized_perf = (stock_performance - np.min(stock_performance)) / (np.max(stock_performance) - np.min(stock_performance))
        weights = normalized_perf + 0.5  # Add base weight to avoid zeros
    else:
        weights = np.ones(N)
    
    # Add some randomness
    weights += np.random.random(N) * 0.5
    
    # Apply constraints
    weights = np.minimum(weights, w_max * np.sum(weights))
    weights = np.maximum(weights, w_min)
    
    # Normalize
    if np.sum(weights) > 0:
        weights /= np.sum(weights)
    else:
        weights = np.ones(N) / N
        
    return weights

# Calculate portfolio metrics for analysis
def calculate_portfolio_metrics(weights):
    portfolio_returns = np.dot(returns_matrix, weights)
    annual_return = np.mean(portfolio_returns) * 252
    annual_std = np.std(portfolio_returns) * np.sqrt(252)
    var = calculate_var(weights, returns_matrix, alpha)
    cvar = calculate_cvar(weights, returns_matrix, alpha)
    reliability_prob = calculate_reliability_constraint(weights, returns_matrix, d_daily)
    
    return {
        'annual_return': annual_return,
        'annual_std': annual_std,
        'var': var,
        'cvar': cvar,
        'reliability_prob': reliability_prob
    }

# Analyze Equal Weight Portfolio
print("\n" + "="*60)
print("EQUAL WEIGHT PORTFOLIO ANALYSIS")
print("="*60)

equal_weights = np.ones(N) / N
equal_metrics = calculate_portfolio_metrics(equal_weights)

print("Equal Weight Portfolio:")
for stock, weight in zip(selected_stocks, equal_weights):
    print(f"{stock}: {weight:.4f}")
print(f"Annual Return: {equal_metrics['annual_return']:.4f}")
print(f"Annual Std Dev: {equal_metrics['annual_std']:.4f}")
print(f"VaR (α={alpha}): {equal_metrics['var']:.4f}")
print(f"CVaR (α={alpha}): {equal_metrics['cvar']:.4f}")
print(f"Reliability P(R≥{d:.3f}): {equal_metrics['reliability_prob']:.4f}")
print(f"Meets Reliability Constraint (≥{beta_1}): {equal_metrics['reliability_prob'] >= beta_1}")

# Analyze Random Portfolios
print("\n" + "="*60)
print("RANDOM PORTFOLIOS ANALYSIS")
print("="*60)

n_random = 10
random_metrics_list = []
for i in range(n_random):
    rw = generate_feasible_weights()
    metrics = calculate_portfolio_metrics(rw)
    random_metrics_list.append(metrics)
    print(f"Random Portfolio {i+1}:")
    print(f"  Annual Return: {metrics['annual_return']:.4f}")
    print(f"  CVaR: {metrics['cvar']:.4f}")
    print(f"  Reliability: {metrics['reliability_prob']:.4f}")
    print(f"  Meets Constraint: {metrics['reliability_prob'] >= beta_1}")

avg_random_return = np.mean([m['annual_return'] for m in random_metrics_list])
avg_random_cvar = np.mean([m['cvar'] for m in random_metrics_list])
avg_random_reliability = np.mean([m['reliability_prob'] for m in random_metrics_list])
print(f"\nAverage Random Return: {avg_random_return:.4f}")
print(f"Average Random CVaR: {avg_random_cvar:.4f}")
print(f"Average Random Reliability: {avg_random_reliability:.4f}")

# Setup DEAP for reliability-based optimization
if 'FitnessMin' in dir(creator):
    del creator.FitnessMin
if 'FitnessMulti' in dir(creator):
    del creator.FitnessMulti
if 'Individual' in dir(creator):
    del creator.Individual

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # Minimize both objectives
creator.create("Individual", list, fitness=creator.FitnessMulti)

def init_individual():
    weights = generate_feasible_weights()
    return creator.Individual(weights.tolist())

# GA setup with improved parameters
toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", multi_objective_reliability)
toolbox.register("mate", tools.cxBlend, alpha=0.2)  # Reduced alpha for more conservative crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.01, indpb=0.2)  # Smaller mutation
toolbox.register("select", tools.selNSGA2)

def normalize_individual(individual):
    weights = np.array(individual)
    weights = np.maximum(weights, w_min)
    weights = np.minimum(weights, w_max)
    if np.sum(weights) > 0:
        weights = weights / np.sum(weights)
    else:
        weights = np.ones(len(weights)) / len(weights)
    individual[:] = weights.tolist()
    return individual,

toolbox.register("normalize", normalize_individual)

def main():
    pop_size = 300
    n_gen = 500
    
    print("\n" + "="*60)
    print("RUNNING IMPROVED RELIABILITY-BASED OPTIMIZATION")
    print("="*60)
    print(f"Population Size: {pop_size}")
    print(f"Generations: {n_gen}")
    print(f"Target Return: {d:.3f} (annualized)")
    print(f"Reliability Level: {beta_1:.3f}")
    print(f"CVaR Confidence Level: {alpha:.3f}")
    
    # Initialize population with feasible solutions
    pop = toolbox.population(n=pop_size)
    
    # Ensure all individuals are feasible
    for ind in pop:
        toolbox.normalize(ind)
    
    hof = tools.ParetoFront()  # For multi-objective
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: np.mean([fit[0] for fit in x]))
    stats.register("min", lambda x: np.min([fit[0] for fit in x]))
    
    # Run evolution
    pop, logbook = algorithms.eaMuPlusLambda(
        pop, toolbox, mu=pop_size, lambda_=pop_size*2,
        cxpb=0.6, mutpb=0.3, ngen=n_gen,
        stats=stats, halloffame=hof, verbose=True
    )
    
    # Find best solutions that satisfy constraints
    feasible_solutions = []
    for ind in hof.items:
        weights = np.array(ind)
        weights = weights / np.sum(weights)  # Ensure normalized
        metrics = calculate_portfolio_metrics(weights)
        
        if (metrics['reliability_prob'] >= beta_1 and 
            metrics['annual_return'] >= d):
            feasible_solutions.append((ind, metrics))
    
    if not feasible_solutions:
        print("\nNo fully feasible solutions found. Selecting best compromise...")
        # Select solution with highest reliability among those meeting return constraint
        candidates = []
        for ind in hof.items:
            weights = np.array(ind)
            weights = weights / np.sum(weights)
            metrics = calculate_portfolio_metrics(weights)
            if metrics['annual_return'] >= d * 0.9:  # Within 90% of target
                candidates.append((ind, metrics))
        
        if candidates:
            best_individual, best_metrics = max(candidates, 
                                              key=lambda x: x[1]['reliability_prob'])
        else:
            best_individual = hof.items[0]
            best_weights = np.array(best_individual) / np.sum(np.array(best_individual))
            best_metrics = calculate_portfolio_metrics(best_weights)
    else:
        # Select solution with best return among feasible ones
        best_individual, best_metrics = max(feasible_solutions, 
                                          key=lambda x: x[1]['annual_return'])
    
    best_weights = np.array(best_individual)
    best_weights = best_weights / np.sum(best_weights)  # Final normalization
    
    print("\n" + "="*60)
    print("OPTIMIZED RELIABILITY-BASED PORTFOLIO RESULTS")
    print("="*60)
    print("Optimized Portfolio Weights:")
    for stock, weight in zip(selected_stocks, best_weights):
        print(f"{stock}: {weight:.4f}")
    
    print(f"\nPortfolio Performance:")
    print(f"Annual Return: {best_metrics['annual_return']:.4f}")
    print(f"Annual Std Dev: {best_metrics['annual_std']:.4f}")
    print(f"VaR (α={alpha}): {best_metrics['var']:.4f}")
    print(f"CVaR (α={alpha}): {best_metrics['cvar']:.4f}")
    print(f"Reliability P(R≥{d:.3f}): {best_metrics['reliability_prob']:.4f}")
    print(f"Sum of Weights: {np.sum(best_weights):.4f}")
    
    print(f"\nConstraint Satisfaction:")
    print(f"Return Target (≥{d:.3f}): {'✓' if best_metrics['annual_return'] >= d else '✗'} ({best_metrics['annual_return']:.4f})")
    print(f"Reliability Constraint (≥{beta_1}): {'✓' if best_metrics['reliability_prob'] >= beta_1 else '✗'} ({best_metrics['reliability_prob']:.4f})")
    print(f"Weight Bounds Constraint: {'✓' if all(w_min <= w <= w_max for w in best_weights) else '✗'}")
    print(f"Weight Sum Constraint: {'✓' if abs(np.sum(best_weights) - 1.0) < 1e-6 else '✗'}")
    
    # Comparison with benchmarks
    print(f"\nPerformance Comparison:")
    print(f"Optimized vs Equal Weight Return: {best_metrics['annual_return']:.4f} vs {equal_metrics['annual_return']:.4f}")
    print(f"Optimized vs Average Random Return: {best_metrics['annual_return']:.4f} vs {avg_random_return:.4f}")
    print(f"Optimized vs Equal Weight Reliability: {best_metrics['reliability_prob']:.4f} vs {equal_metrics['reliability_prob']:.4f}")

if __name__ == "__main__":
    main()

Selected stocks: ['INFY.NS', 'RELIANCE.NS', 'ITC.NS', 'BHARTIARTL.NS', 'TITAN.NS', 'HDFCBANK.NS', 'KOTAKBANK.NS', 'TCS.NS', 'AXISBANK.NS', 'NESTLEIND.NS']


[*********************100%***********************]  10 of 10 completed


Returns shape: (739, 10)
Number of scenarios (T): 739
Number of assets (N): 10

Annualized Mean Returns:
INFY.NS: 0.2158
RELIANCE.NS: 0.3778
ITC.NS: 0.1859
BHARTIARTL.NS: 0.0876
TITAN.NS: 0.2187
HDFCBANK.NS: 0.0622
KOTAKBANK.NS: 0.1331
TCS.NS: 0.0830
AXISBANK.NS: 0.0546
NESTLEIND.NS: 0.1996

Optimization Parameters:
Target Annual Return: 0.030
Daily Target Return: 0.000119
Reliability Level: 0.800

EQUAL WEIGHT PORTFOLIO ANALYSIS
Equal Weight Portfolio:
INFY.NS: 0.1000
RELIANCE.NS: 0.1000
ITC.NS: 0.1000
BHARTIARTL.NS: 0.1000
TITAN.NS: 0.1000
HDFCBANK.NS: 0.1000
KOTAKBANK.NS: 0.1000
TCS.NS: 0.1000
AXISBANK.NS: 0.1000
NESTLEIND.NS: 0.1000
Annual Return: 0.1618
Annual Std Dev: 0.1159
VaR (α=0.05): 0.0114
CVaR (α=0.05): 0.0151
Reliability P(R≥0.030): 0.5223
Meets Reliability Constraint (≥0.8): False

RANDOM PORTFOLIOS ANALYSIS
Random Portfolio 1:
  Annual Return: 0.1908
  CVaR: 0.0150
  Reliability: 0.5196
  Meets Constraint: False
Random Portfolio 2:
  Annual Return: 0.1944
  CVaR: 0.0149